In [2]:
import numpy as np
from scipy.io import loadmat
import os
import glob
import re
import sys
from six.moves import cPickle as pickle

print("import done")

import done


In [ ]:
#get segment from 'patient_segment_result.mat' for file name ordering
def natural_key(string_):
    return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string_)]

#convert .mat data into ndarray and compress it usingle pickle
def pickle_data(files_pickle, name_pickle):
    print('Reading file ' + files_pickle[0] + ' to ' + files_pickle[-1])
    nrfiles = len(files_pickle)
    placeholder_data = np.ndarray(shape=(nrfiles, 400*10*60, 16), dtype=np.float32)
    placeholder_seq = np.ndarray(shape=(nrfiles), dtype=np.int32)
    for idx, fl in enumerate(files_pickle):
        #open mat file
        mat = loadmat(fl)
        #convert from ndarray object to ndarray
        names = mat['dataStruct'].dtype.names
        ndata = {n: mat['dataStruct'][n][0, 0] for n in names}
        #detect if columns out of order
        if not all(x<y for x, y in zip(ndata['channelIndices'][0], ndata['channelIndices'][0][1:])):
            print('WARNING: Columns out of order in file' + fl)
        placeholder_data[idx, :, :] = ndata['data']
        placeholder_seq[idx] = ndata['sequence']
    #bundle data in a dictionary 
    placeholder = {'data': placeholder_data, 'seq': placeholder_seq}
    
    try:
        with open(name_pickle, 'wb') as f:
            print('Pickling to ' + name_pickle)
            pickle.dump(placeholder, f, pickle.HIGHEST_PROTOCOL)
    except Exception as e:
        print('Unable to save data to' + name_pickle + ' :', e)

nrfiles_pickle = 45
        
def dataset_patient(folder_pat, result):
    #get files
    files = sorted(glob.glob(folder_pat + "/*" + result + ".mat"), key=natural_key)
    files = [files[i:i + nrfiles_pickle] for i in xrange(0, len(files), nrfiles_pickle)]
    #concatenate remainder if size too small
    if (len(files[-1]) < 4):
        files[-2].extend(files[-1])
        del files[-1]
    #create pickle files
    for idx, fl in enumerate(files):
        picklefile = folder_pat.split('/')[1] + '_res' + result + '_idx' + str(idx) + '.pickle'
        if (os.path.exists(picklefile)):
            print(picklefile + '  already present. Skip pickling.')
        else:
            pickle_data(fl, picklefile)
    
dataset_patient('./train_1', '1')
dataset_patient('./train_1', '0')

print('done')

train_1_res1_idx0.pickle  already present. Skip pickling.
train_1_res1_idx1.pickle  already present. Skip pickling.
train_1_res1_idx2.pickle  already present. Skip pickling.
train_1_res1_idx3.pickle  already present. Skip pickling.
Reading file ./train_1/1_1_0.mat to ./train_1/1_45_0.mat
Pickling to train_1_res0_idx0.pickle
Reading file ./train_1/1_46_0.mat to ./train_1/1_90_0.mat
Pickling to train_1_res0_idx1.pickle
Reading file ./train_1/1_91_0.mat to ./train_1/1_135_0.mat
Pickling to train_1_res0_idx2.pickle
Reading file ./train_1/1_136_0.mat to ./train_1/1_180_0.mat
Pickling to train_1_res0_idx3.pickle
Reading file ./train_1/1_181_0.mat to ./train_1/1_225_0.mat
Pickling to train_1_res0_idx4.pickle
Reading file ./train_1/1_226_0.mat to ./train_1/1_270_0.mat
Pickling to train_1_res0_idx5.pickle
Reading file ./train_1/1_271_0.mat to ./train_1/1_315_0.mat
Pickling to train_1_res0_idx6.pickle
Reading file ./train_1/1_316_0.mat to ./train_1/1_360_0.mat
Pickling to train_1_res0_idx7.pickl

In [20]:
with open('train_1_res1_idx0.pickle', 'rb') as f:
    data = pickle.load(f)
    print(data['data'].shape)
    print(data['seq'].shape)

(37, 240000, 16)


In [28]:
x = np.array([[1, 1], [2, 2], [1,2]])

print(x)

print(x[0])

[[1 1]
 [2 2]
 [1 2]]
[1 1]
